<a href="https://colab.research.google.com/github/asasinater1234/mAId/blob/main/Stage_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
HOME = os.getcwd()
print(HOME)

/content


In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

# prevent ultralytics from tracking your activity
!yolo settings sync=False

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 21.5/107.7 GB disk)


In [ ]:
# Install and import Roboflow
!pip install roboflow -q

from roboflow import Roboflow
import os

# Download the dataset
rf = Roboflow(api_key="lGEVfyWp66tashZarQZu")
project = rf.workspace("labrats").project("sutd_stage2-glzmd")
version = project.version(1)

# Download in YOLO format (which includes data.yaml)
dataset = version.download("yolov8")

print(f"✅ Dataset downloaded to: {dataset.location}")

# Save the location for later use
DATASET_PATH = dataset.location
print(f"📂 Dataset path saved: {DATASET_PATH}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.7 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to SUTD_STAGE2-1 in yolov8:: 100%|██████████| 110/110 [00:00<00:00, 6797.30it/s]

✅ Dataset downloaded to: /content/SUTD_STAGE2-1
📂 Dataset path saved: /content/SUTD_STAGE2-1


In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

# prevent ultralytics from tracking your activity
!yolo settings sync=False

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 21.7/107.7 GB disk)


In [ ]:
!pip install -q roboflow timm albumentations scikit-learn pyyaml tqdm torch torchvision

In [ ]:
!mkdir -p ~/datasets
%cd ~/datasets

!pip install roboflow==1.1.48 --quiet

from roboflow import Roboflow

rf = Roboflow(api_key="lGEVfyWp66tashZarQZu")
project = rf.workspace("labrats").project("sutd_stage2-glzmd")

dataset = project.version(1).download("yolov8")  # ✅ should create data.yaml
print(dataset.location)


/root/datasets
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 1.7 MB/s eta 0:00:00
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to SUTD_STAGE2-1 in yolov8:: 100%|██████████| 110/110 [00:00<00:00, 5591.46it/s]

/root/datasets/SUTD_STAGE2-1


In [ ]:
# Go to datasets
!mkdir -p ~/datasets
%cd ~/datasets

# Install roboflow
!pip install roboflow==1.1.48 --quiet

from roboflow import Roboflow
import os, shutil, glob, random

# Download YOLOv8 dataset
rf = Roboflow(api_key="lGEVfyWp66tashZarQZu")
project = rf.workspace("labrats").project("sutd_stage2-glzmd")
dataset = project.version(1).download("yolov8")

data_dir = dataset.location  # e.g. /root/datasets/SUTD_STAGE2-1
print("Dataset location:", data_dir)

# -----------------------------
# Create valid split if missing
# -----------------------------
train_img_dir = os.path.join(data_dir, "train", "images")
train_lbl_dir = os.path.join(data_dir, "train", "labels")

valid_img_dir = os.path.join(data_dir, "valid", "images")
valid_lbl_dir = os.path.join(data_dir, "valid", "labels")

if not os.path.exists(valid_img_dir):
    os.makedirs(valid_img_dir, exist_ok=True)
    os.makedirs(valid_lbl_dir, exist_ok=True)

    imgs = glob.glob(os.path.join(train_img_dir, "*.jpg")) + glob.glob(os.path.join(train_img_dir, "*.png"))
    random.shuffle(imgs)

    val_ratio = 0.2
    val_count = int(len(imgs) * val_ratio)

    val_imgs = imgs[:val_count]

    for img_path in val_imgs:
        base = os.path.splitext(os.path.basename(img_path))[0]
        lbl_path = os.path.join(train_lbl_dir, base + ".txt")

        # move image
        shutil.move(img_path, os.path.join(valid_img_dir, os.path.basename(img_path)))

        # move label
        if os.path.exists(lbl_path):
            shutil.move(lbl_path, os.path.join(valid_lbl_dir, os.path.basename(lbl_path)))

    print(f"✅ Created valid split: moved {val_count} images to valid/")

# -----------------------------
# Train YOLOv8
# -----------------------------
%cd ~
!yolo task=detect mode=train model=yolov8s.pt data="{data_dir}/data.yaml" epochs=10 imgsz=800 plots=True


/root/datasets
loading Roboflow workspace...
loading Roboflow project...
Dataset location: /root/datasets/SUTD_STAGE2-1
/root
New https://pypi.org/project/ultralytics/8.4.7 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.103 🚀 Python-3.12.12 torch-2.9.0+cpu CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/root/datasets/SUTD_STAGE2-1/data.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=800, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stre